# Plot ELG chains
This jupyter notebook contains some code to make plots of the bestfits and contours for the abacus cubic ELG fits to both the mock mean and individual realisations.

In [ ]:
# Import some necessary modules
import os
import sys
import pickle
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from chainconsumer import ChainConsumer     

sys.path.append("../../Barry/")     # Change this so that it points to where you have Barry installed

from barry.samplers import DynestySampler
from barry.config import setup
from barry.models import PowerBeutler2017, CorrBeutler2017
from barry.datasets.dataset_power_spectrum import PowerSpectrum_DESI_KP4
from barry.datasets.dataset_correlation_function import CorrelationFunction_DESI_KP4
from barry.fitter import Fitter
from barry.models.model import Correction # class for applying corrections to the likelihood function 
from barry.utils import weighted_avg_and_cov # function for getting avg and covariance 

#print(sys.path)
# Read in the fitter class to get all the info on the fit
#pfn = "../plots/desi_kp4_abacus_cubic_ELG/output/desi_kp4_abacus_cubic_ELG.fitter.pkl"

# omega_matter_free = False 

# neff_free = False 

phase_shift_free = False 

pfn = "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG_isotropic/output/desi_kp4_abacus_cubic_ELG_isotropic.fitter.pkl"

# if omega_matter_free: 
#     pfn = "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG_omfree/output/desi_kp4_abacus_cubic_ELG_omfree.fitter.pkl"

# if neff_free: 
#     pfn = "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG_nefffree/output/desi_kp4_abacus_cubic_ELG_nefffree.fitter.pkl"
    
if phase_shift_free: 
    pfn = "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG_phaseshiftfree_isotropic/output/desi_kp4_abacus_cubic_ELG_phaseshiftfree_isotropic.fitter.pkl"
    

    
with open(pfn, 'rb') as pickle_file:
    fitter = pickle.load(pickle_file)
    
#print(fitter.load())

This code segment reads in the chains, plots the bestfit model vs. data and prepares stuff for contour plots and summary files

In [ ]:
logging.info("Creating plots")

fitname = []
datanames = ["CV_Xi_Prerecon", "CV_Xi_Recon", "CV_Pk_Prerecon", "CV_Pk_Recon"]
c = [ChainConsumer(), ChainConsumer(), ChainConsumer(), ChainConsumer()]

# Loop over all the chains
stats = {k: [] for k in datanames}
output = {k: [] for k in datanames}
for posterior, weight, chain, evidence, model, data, extra in fitter.load():

    #print(extra['name'])
    #print(model.get_labels(), chain.shape)
    # Get the realisation number and redshift bin
    
    #if "CV" not in extra["name"] or "Pk" not in extra["name"]:
    #    continue
    
    if "Pk" not in extra["name"]:
       continue 
    #print(extra)
        
    recon_bin = 0 if "Prerecon" in extra["name"] else 1
    data_bin = 0 if "Xi" in extra["name"] else 1
    realisation = str(extra["name"].split()[-1]) if "realisation" in extra["name"] else "mean"
    chain_bin = int(2.0 * data_bin + recon_bin)
    print(extra["name"], chain_bin, realisation)
    #print(extra['name'])
    #print(model.get_labels(), chain.shape)
    # Store the chain in a dictionary with parameter names
    df = pd.DataFrame(chain, columns=model.get_labels())
    
    # Compute alpha_par and alpha_perp for each point in the chain
    # alpha_par, alpha_perp = model.get_alphas(df["$\\alpha$"].to_numpy(), df["$\\epsilon$"].to_numpy())
    # df["$\\alpha_\\parallel$"] = alpha_par
    # df["$\\alpha_\\perp$"] = alpha_perp
                                                        
    # Get the MAP (maximum a posteriori) point and set the model up at this point
    model.set_data(data)
    r_s = model.camb.get_data()["r_s"]
    max_post = posterior.argmax()
    params = df.loc[max_post]
    params_dict = model.get_param_dict(chain[max_post])
    for name, val in params_dict.items():
        model.set_default(name, val)

    # Get some useful properties of the fit, and plot the MAP model against the data if it's the mock mean or realisation 10 (chosen randomly!)
    display = True if realisation == "mean" or realisation == "10" else False
    new_chi_squared, dof, bband, mods, smooths = model.plot(params_dict, display=display)

    # Add the chain or MAP to the Chainconsumer plots
    extra.pop("realisation", None)
    if realisation == "mean":
        c[chain_bin].add_chain(df, weights=weight, **extra, plot_contour=True, plot_point=False, show_as_1d_prior=False)
    else:
        c[chain_bin].add_marker(params, **extra, marker_size=80)

    # Compute some summary statistics and add them to a dictionary
    
    params = [
                #"$\\alpha_\\parallel$",
                #"$\\alpha_\\perp$",
                "$\\alpha$",
                #"$\\Sigma_{nl,||}$",
                #"$\\Sigma_{nl,\\perp}$",
                "$\\Sigma_{nl}$"
            ]
    
#     if omega_matter_free: 
#         params.append("$\\Omega_m$")
        
#     if neff_free:
#         params.append("$N_{\mathrm{eff}}$")
        
    if phase_shift_free: 
        params.append("$\phi$")
    
    mean, cov = weighted_avg_and_cov(
        df[
            params
        ],
        weight,
        axis=0,
    )
    
#     if omega_matter_free: ###################################
        
#         corr_alpha_para_perp = cov[1, 0] / np.sqrt(cov[0, 0] * cov[1, 1])
#         corr_alpha_para_om = cov[4, 0] / np.sqrt(cov[4, 4] * cov[0, 0])
#         corr_alpha_perp_om = cov[4, 1] / np.sqrt(cov[4, 4] * cov[1, 1])
        
#         stats[datanames[chain_bin]].append(
#             [mean[0], mean[1], np.sqrt(cov[0, 0]), np.sqrt(cov[1, 1]), 
#             corr_alpha_para_perp, mean[4], np.sqrt(cov[4,4]), corr_alpha_para_om, corr_alpha_perp_om, new_chi_squared, mean[2], mean[3]]
#         )

#         output[datanames[chain_bin]].append(
#             f"{realisation:s}, {mean[0]:6.4f}, {mean[1]:6.4f}, {mean[2]:6.4f}, {mean[3]:6.4f}, {np.sqrt(cov[0, 0]):6.4f}, {np.sqrt(cov[1, 1]):6.4f}, {corr_alpha_para_perp:7.3f}, {mean[4]:6.4f}, {np.sqrt(cov[4, 4]):6.4f},{corr_alpha_para_om:7.3f}, {corr_alpha_perp_om:7.3f}, {r_s:7.3f}, {new_chi_squared:7.3f}, {dof:4d}"
#         )
        
        
#     if neff_free: ###################################
        
#         corr_alpha_para_perp = cov[1, 0] / np.sqrt(cov[0, 0] * cov[1, 1])
#         corr_alpha_para_neff = cov[4, 0] / np.sqrt(cov[4, 4] * cov[0, 0])
#         corr_alpha_perp_neff = cov[4, 1] / np.sqrt(cov[4, 4] * cov[1, 1])
        
#         stats[datanames[chain_bin]].append(
#             [mean[0], mean[1], np.sqrt(cov[0, 0]), np.sqrt(cov[1, 1]), 
#             corr_alpha_para_perp, mean[4], np.sqrt(cov[4,4]), corr_alpha_para_neff, corr_alpha_perp_neff, new_chi_squared, mean[2], mean[3]]
#         )

#         output[datanames[chain_bin]].append(
#             f"{realisation:s}, {mean[0]:6.4f}, {mean[1]:6.4f}, {mean[2]:6.4f}, {mean[3]:6.4f}, {np.sqrt(cov[0, 0]):6.4f}, {np.sqrt(cov[1, 1]):6.4f}, {corr_alpha_para_perp:7.3f}, {mean[4]:6.4f}, {np.sqrt(cov[4, 4]):6.4f},{corr_alpha_para_neff:7.3f}, {corr_alpha_perp_neff:7.3f}, {r_s:7.3f}, {new_chi_squared:7.3f}, {dof:4d}"
#         )
        
    
    # below here needs fixing 
    ########################################################################################
    ########################################################################################
    ########################################################################################
    ########################################################################################
    
    if phase_shift_free: ################################### 
        
        corr_alpha_phaseshift = cov[0, 2] / np.sqrt(cov[0, 0] * cov[2, 2])
        
        stats[datanames[chain_bin]].append(
            [mean[0], np.sqrt(cov[0, 0]), mean[2], np.sqrt(cov[2, 2]), 
            corr_alpha_phaseshift, new_chi_squared, mean[1]]
        )

        output[datanames[chain_bin]].append(
            f"{realisation:s}, {mean[0]:6.4f}, {mean[1]:6.4f}, {mean[2]:6.4f}, {np.sqrt(cov[0, 0]):6.4f}, {np.sqrt(cov[2, 2]):6.4f}, {corr_alpha_phaseshift:7.3f}, {r_s:7.3f}, {new_chi_squared:7.3f}, {dof:4d}"
        )
        
        
    else: ###################################

        stats[datanames[chain_bin]].append(
            [mean[0], np.sqrt(cov[0, 0]), new_chi_squared, mean[1]]
        )

        output[datanames[chain_bin]].append(
            f"{realisation:s}, {mean[0]:6.4f}, {mean[1]:6.4f}, {np.sqrt(cov[0, 0]):6.4f}, {r_s:7.3f}, {new_chi_squared:7.3f}, {dof:4d}"
        )

Plot the contour plots and output the summary statistics

In [ ]:
def plot_errors(stats, #omega_matter_free=False, neff_free=False, 
                phase_shift_free=False, suffix=''):
    
    nstats = len(stats)
    means = np.mean(stats, axis=0)
    covs = np.cov(stats, rowvar=False)
    #corr = covs[0, 1] / np.sqrt(covs[0, 0] * covs[1, 1])

    #labels = [r"$\sigma_{\alpha,||}$", r"$\sigma_{\alpha,\perp}$", r"$\rho(\alpha_{||},\alpha_{\perp})$"] 
    labels = [r"$\sigma_{\alpha}$"]
    colors = ["r"]#, "b", "g"]
    
    #print(np.array(stats).T.shape)
    
#     if omega_matter_free:
#         labels.append(r"$\sigma_{\Omega_m}$")
#         labels.append(r"$\rho(\alpha_{||},\Omega_m)$")
#         labels.append(r"$\rho(\alpha_{\perp},\Omega_m)$")
#         colors.append("orange")
#         colors.append("grey")
#         colors.append("yellow")
        
#     if neff_free:
#         labels.append(r"$\sigma_{N_{\mathrm{eff}}}$")
#         labels.append(r"$\rho(\alpha_{||},N_{\mathrm{eff}})$")
#         labels.append(r"$\rho(\alpha_{\perp},N_{\mathrm{eff}})$")
#         colors.append("orange")
#         colors.append("grey")
#         colors.append("yellow")
        
    if phase_shift_free: 
        labels.append(r"$\sigma_{\phi}$")
        labels.append(r"$\rho(\alpha,\phi)$")
        #labels.append(r"$\rho(\alpha_{\perp},\phi)$")
        colors.append("b")
        colors.append("g")
        #colors.append("yellow")

    fig, axes = plt.subplots(figsize=(7, 2), nrows=1, ncols=len(labels), sharey=True, squeeze=False)
    #if omega_matter_free or neff_free or phase_shift_free:
    #    plt.subplots_adjust(left=0.1, top=0.95, bottom=0.05, right=2.0, hspace=0.3)
    #else:
    plt.subplots_adjust(left=0.1, top=0.95, bottom=0.05, right=0.95, hspace=0.3)
    
    range_vals = [0]#[2,3,4]
    range_means = [0]#[2,3,4]
    stds_arr = [np.sqrt(covs[0, 0])]#, np.sqrt(covs[1, 1]), corr]
    
    #if omega_matter_free or neff_free or 
    if phase_shift_free: 
        range_vals = [0,1,6] 
        stds_arr = [
                   np.sqrt(covs[0, 0]), 
                   np.sqrt(covs[1, 1]), 
                   covs[0, 6] / np.sqrt(covs[0, 0] * covs[6, 6]),
                   #np.sqrt(covs[5, 5]),
                   #covs[0, 5] / np.sqrt(covs[0, 0] * covs[5, 5]),
                   #covs[5, 1] / np.sqrt(covs[5, 5] * covs[1, 1])
                   ]
        range_means = [0,1,6] 
    
    for ax, vals, avgs, stds, l, c in zip(axes.T, np.array(stats).T[range_vals,:], means[range_means], stds_arr, labels, colors):

        ax[0].hist(vals, 10, color=c, histtype="stepfilled", alpha=0.2, density=False, zorder=0)
        ax[0].hist(vals, 10, color=c, histtype="step", alpha=1.0, lw=1.3, density=False, zorder=1)
        ax[0].axvline(avgs, color="k", ls="--", zorder=2)
        ax[0].axvline(stds, color="k", ls=":", zorder=2)
        ax[0].set_xlabel(l)
        
    axes[0, 0].set_ylabel(r"$N_{\mathrm{mocks}}$")

    plt.savefig("plots/desi_kp4_abacus_cubic_ELG"+suffix+"/output/histplot.png", bbox_inches='tight')
    plt.show()

    return nstats, means, covs, corr

truth = {"$\\alpha$": 1.0, }#"$\\epsilon$": 0, "$\\alpha_\\perp$": 1.0, "$\\alpha_\\parallel$": 1.0}
# if omega_matter_free: 
#     truth["$\\Omega_m$"] = 0.3
# if neff_free:
#     truth["$N_{\mathrm{eff}}$"] = 3.044
# if phase_shift_free:
    truth["$\phi$"] = 1.0

parameters = [names for names in truth]

suffix = '_isotropic'

# if omega_matter_free: 
#     suffix = '_omfree'
# elif neff_free:
#     suffix = '_nefffree'
elif phase_shift_free:
    suffix = '_phaseshiftfree_isotropic'

for chain_bin in range(len(c)):
    
    #if datanames[chain_bin] != 'CV_Pk_Recon':
    #    continue 
        
    if 'Pk' not in datanames[chain_bin]:
        continue 
        
    #print(datanames[chain_bin])
    
    #print(c[chain_bin])
        
    c[chain_bin].plotter.plot(
    truth=truth,
        parameters=(parameters),
        legend=False,
        display=True,
        figsize=(6,6), 
        filename="plots/desi_kp4_abacus_cubic_ELG" + suffix + "/output/contourplot.png",
        
    )
    
    # Plot histograms of the errors and r_off
    nstats, means, covs, corr = plot_errors(stats[datanames[chain_bin]],
                                            #omega_matter_free=omega_matter_free,
                                            #neff_free=neff_free,
                                            phase_shift_free=phase_shift_free,
                                            suffix=suffix)
    
    
            
    if phase_shift_free: ###################################
        
        

        output[datanames[chain_bin]].append(
            f"{realisation:s}, {mean[0]:6.4f}, {mean[1]:6.4f}, {mean[2]:6.4f}, {np.sqrt(cov[0, 0]):6.4f}, {np.sqrt(cov[2, 2]):6.4f}, {corr_alpha_phaseshift:7.3f}, {r_s:7.3f}, {new_chi_squared:7.3f}, {dof:4d}"
        )
        
        
        with open("plots/desi_kp4_abacus_cubic_ELG_phaseshiftfree/output/Barry_fit_" + datanames[chain_bin] + ".txt", "w") as f:
            f.write(
                "# Realisation, alpha, Sigma_nl, phi, sigma_alpha, sigma_phi, corr_phi_alpha, rd_of_template, bf_chi2, dof\n"
                )
            for l in output[datanames[chain_bin]]:
                f.write(l + "\n")

            # And now the average of all the individual realisations
            f.write("# ---------------------------------------------------\n")
            f.write(
                "# <alpha>, <sigma_alpha>, <phi>, <sigma_phi>, <corr_alpha_phaseshift>, <sigma_nl>, <bf_chi2>, std_alpha, std_phi, corr_alpha_phaseshift\n"
                )
            f.write(
                f"{means[0]:6.4f}, {means[1]:6.4f}, {means[2]:6.4f}, {means[3]:6.4f}, {means[4]:6.4f}, {means[6]:6.4f}, {means[5].6.4f}, {np.sqrt(covs[0, 0]):6.4f}, {np.sqrt(covs[2, 2]):6.4f}, {(covs[0, 2] / np.sqrt(covs[0, 0] * covs[2, 2]))}\n"
                )
        
    else:###################################
            
        with open("plots/desi_kp4_abacus_cubic_ELG/output/Barry_fit_" + datanames[chain_bin] + ".txt", "w") as f:
            f.write(
                "# Realisation, alpha, Sigma_nl, sigma_alpha, rd_of_template, bf_chi2, dof\n"
                )
            for l in output[datanames[chain_bin]]:
                f.write(l + "\n")

                
            # And now the average of all the individual realisations
            f.write("# ---------------------------------------------------\n")
            f.write(
                "# <alpha>, <sigma_alpha>, <bf_chi2>, <Sigma_nl>, std_alpha, std_sigma_nl \n"
                )
            f.write(
                f"{means[0]:6.4f}, {means[1]:6.4f}, {means[2]:6.4f}, {means[3]:6.4f}, {np.sqrt(covs[0, 0]):6.4f}, {np.sqrt(covs[1, 1]):6.4f}\n"
                )